In [1]:
import time
inicio=time.time()

In [2]:
import numpy as np
import pandas as pd

In [3]:
imp_fert=pd.read_csv('~/git/BD/temp/imp_fert_quanti_g_ncm_uf.csv').sort_values(by=['CO_ANO','CO_MES']).reset_index(drop=True)

In [4]:
imp_fert=imp_fert.loc[:,:'VL_FOB']

In [5]:
filt_ncm=imp_fert.groupby('CO_NCM').KG_LIQUIDO.sum().sort_values(ascending=False).reset_index().CO_NCM.values[:1]
filt_ncm

array(['31042090-hi'], dtype=object)

In [6]:
filt_uf=imp_fert.groupby('SG_UF_NCM').KG_LIQUIDO.sum().sort_values(ascending=False).reset_index().SG_UF_NCM.values[:1]
filt_uf

array(['PR'], dtype=object)

In [7]:
len_ncms=len(filt_ncm)

In [8]:
imp_fert=imp_fert[(imp_fert.SG_UF_NCM.isin(filt_uf))&(imp_fert.CO_NCM.isin(filt_ncm))]

In [9]:
%run '~/git/sparse_matrix.ipynb'
imp_fert=sparse_matrix(imp_fert)

*** sparse_matrix(imp_fert) available ***


In [10]:
%run '~/git/multi.ipynb'
imp_fert=multi(imp_fert)
imp_fert=pd.concat([imp_fert.iloc[:,:7] , imp_fert.iloc[:,7:]],axis=1)

*** multi(imp_fert) function available ***


In [11]:
cols_ad=imp_fert.iloc[:,7:].columns.tolist()

In [16]:
max_depth=5
from sklearn.ensemble import RandomForestClassifier as RFC
RFC=RFC(max_depth=max_depth)
from sklearn.ensemble import RandomForestRegressor as RFR
RFR=RFR(max_depth=max_depth)
from sklearn.linear_model import LinearRegression as LR
LR=LR()
REG=RFR

In [17]:
%run '~/git/prev_ncm_uf.ipynb'
prev=prev(imp_fert,REG)

*** prev(ano_mes_ncm_uf, REG) function available ***


In [18]:
%run '~/git/score_ncm_uf.ipynb'
score=score(prev)

*** score(ano_mes_ncm_uf) function available ***


In [20]:
score=score.assign(len_ncms=len_ncms).assign(cols_ad=str(cols_ad)).assign(REG=str(REG))

In [21]:
score.to_csv('~/git/BD/temp/score.csv',index=False)

In [22]:
fim=time.time()
d=fim-inicio
print(f'\nDuração total:{round(d/60,1)} minutos.\n')


Duração total:4.4 minutos.



In [23]:
score[['accuracy','precision','recall','r2']].mean()

accuracy     0.968795
precision    0.979942
recall       0.987698
r2          -0.413262
dtype: float64

In [24]:
score

,CO_MES,CO_NCM,SG_UF_NCM,accuracy,precision,recall,r2,len_ncms,len_cols,REG
0,3,31042090-hi,PR,0.954545,0.954545,1.000000,-0.191355,1,"['31042090-hi~PR_kg_ad_1', '31042090-hi~PR_fob...",RandomForestRegressor(max_depth=5)
0,4,31042090-hi,PR,0.909091,0.952381,0.952381,-0.353283,1,"['31042090-hi~PR_kg_ad_1', '31042090-hi~PR_fob...",RandomForestRegressor(max_depth=5)
0,5,31042090-hi,PR,0.904762,0.950000,0.950000,0.109188,1,"['31042090-hi~PR_kg_ad_1', '31042090-hi~PR_fob...",RandomForestRegressor(max_depth=5)
0,6,31042090-hi,PR,0.904762,0.950000,0.950000,-0.731100,1,"['31042090-hi~PR_kg_ad_1', '31042090-hi~PR_fob...",RandomForestRegressor(max_depth=5)
0,7,31042090-hi,PR,1.000000,1.000000,1.000000,-0.486820,1,"['31042090-hi~PR_kg_ad_1', '31042090-hi~PR_fob...",RandomForestRegressor(max_depth=5)
0,8,31042090-hi,PR,1.000000,1.000000,1.000000,-0.339711,1,"['31042090-hi~PR_kg_ad_1', '31042090-hi~PR_fob...",RandomForestRegressor(max_depth=5)
0,9,31042090-hi,PR,1.000000,1.000000,1.000000,-0.562522,1,"['31042090-hi~PR_kg_ad_1', '31042090-hi~PR_fob...",RandomForestRegressor(max_depth=5)
0,10,31042090-hi,PR,1.000000,1.000000,1.000000,-0.763273,1,"['31042090-hi~PR_kg_ad_1', '31042090-hi~PR_fob...",RandomForestRegressor(max_depth=5)
0,11,31042090-hi,PR,1.000000,1.000000,1.000000,-0.311933,1,"['31042090-hi~PR_kg_ad_1', '31042090-hi~PR_fob...",RandomForestRegressor(max_depth=5)
0,12,31042090-hi,PR,1.000000,1.000000,1.000000,-0.154567,1,"['31042090-hi~PR_kg_ad_1', '31042090-hi~PR_fob...",RandomForestRegressor(max_depth=5)
